This notebook will demonstrate a iterative training and deployment process of an Imagenet-based image recognition 

First, we define all of our parameters that will be used in the notebook

In [ ]:
# Setup our Sagemaker configuration tunables
aws_region = 'us-east-1'

# We will use the Keras mobilenet V2 model
model_framework = 'keras'
model_name = 'demo-' + model_framework
model_basename = 'mobilenet_v2'
model_data_shape = '{"input_1":[1,3,224,224]}'
packaged_model_name = model_framework + "-model"
packaged_model_version = "1.0"

# Input images
input_image_1 = 'keras.bmp'
input_image_2 = 'darknet.bmp'
input_image_3 = 'plane.jpeg'

# Our Pelion Edge Gateway is a Nvidia Jetson Xavier
target_device = 'jetson_xavier'

# Set our Pelion API Configuration Here
api_key = 'ak_2MDE2ZmZjZmQwOTgxNGE2OGYxNmFlODQ0MDAwMDAwMDA0176d3f035509609f897638900000000Negxfm3p2PwYYEkqN1VKjqxMlM07HOzA'
device_id = '0178a7b9f49516522fada71600300000'
endpoint_api = 'api.' + aws_region + '.mbedcloud.com'   # This is optional and the default

Next we import some additional packages into our python environment

In [ ]:
# Lets install some image utilities
! pip install ipympl

# We also need to install the Pelion Sagemaker Controller API
! pip install pelion_sagemaker_controller

# We also install some image utilities for the NB itself
from PIL import Image
import matplotlib.pyplot as plt

# We'll also use time for waiting on predictions to complete...
import time

Next we import the core class for this notebook

In [ ]:
import example_notebook

Next we allocate our notebook class... this will init both Sagamaker as well as the Pelion Controller API

In [ ]:
my_notebook = example_notebook.MyNotebook(api_key, device_id, endpoint_api)

First, lets create a mobilenet V2 model

In [ ]:
import tensorflow as tf
mobilenet_model = model = tf.keras.applications.MobileNetV2()

Lets compile up our model, then package it on S3

In [ ]:
# Compile up for our target Pelion Edge Gateway platform type
job_name = my_notebook.compile_model(mobilenet_model, target_device, model_basename, model_data_shape, model_framework)

# Package up and store the compiled model onto S3
model_package = my_notebook.package_model(packaged_model_name, packaged_model_version, job_name)

Next, we (re)load our model since we have just (re)compiled it and (re)packaged it...

In [ ]:
# (re)load the model...
my_notebook.pelion_api.pelion_reload_model(model_name,model_package)

In [ ]:
# list our models
my_notebook.pelion_api.pelion_list_models()

Next we will upload our image from our notebook to S3

In [ ]:
# Lets set our input image filename
local_input_image = "images/" + input_image_1
input_image = Image.open(local_input_image)
plt.imshow(input_image,cmap='gray',vmin=0,vmax=255)

# Upload the image to S3
print("Uploading " + input_image_1 + ' to ' + my_notebook.iot_folder + " in S3 bucket " + my_notebook.bucket + '...')
my_notebook.sess.upload_data(local_input_image, my_notebook.bucket, my_notebook.iot_folder)

Now lets do a prediction. We will store the output with the "keras-predicted.data" template back to S3 (a timestamp will be added)

In [ ]:
# Invoke the prediction
my_notebook.pelion_api.pelion_predict(model_name,'s3:///' + input_image_1,'s3:///keras-predicted.data')

# Poll every 5 sec to look for the prediction completion....
while True:
    # Get the last RPC call result...
    print('Prediction Running ...')
    prediction_result = my_notebook.pelion_api.pelion_last_cmd_result();
    if 'details' in prediction_result:
        if 'output' in prediction_result['details']:
            print('Prediction Completed!')
            print("")
            break
    time.sleep(5)

Next we display our results....

In [ ]:
# Prediction results tensor filename
prediction_results_tensor_filename = 'prediction-output.tensor'

# Copy the results back to our notebook
my_notebook.copy_results_to_notebook(prediction_result['details']['output'][0]['url'],prediction_results_tensor_filename)

# Analyze/Display the results
my_notebook.display_results("images/" + input_image_1, prediction_results_tensor_filename)

# Here is our input image again
print("")
print("Here is the input image again:")
plt.imshow(input_image,cmap='gray',vmin=0,vmax=255)